In [113]:
#!pip install modestga --upgrade --quiet
#!pip install deap --upgrade --quiet
import warnings
warnings.filterwarnings("ignore")

In [114]:

import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error

In [115]:
#===========================================================
#                   Read Excel File (Node_Property)
#===========================================================

file = pd.ExcelFile('Result-test.xlsx')
df1=pd.read_excel(file,'Node_Property')
data=df1[df1.columns[:]].to_numpy()
data = np.unique(data,axis=0)
#print ('data: \n',data)
Node_Name= data[:,0]
Memory= data[:,1]                                # Accessible memory size of CNk for running tasks
tsk= data[:,2]                                   # Switch time between two jobs for CNk 
#print(tsk)
#===========================================================
#                   Read Excel File 
#===========================================================

file = pd.ExcelFile('Result-test.xlsx')
df1=pd.read_excel(file,'General-Results')
data=df1[df1.columns[:]].to_numpy()
data = np.unique(data,axis=0)
#print ('data: \n',data)
nodename= data[:,0]
x= data[:,1]
y= data[:,2]


nodename_item =list(dict.fromkeys(nodename))
V = len(nodename_item)  # The number of nodes of the cluster

#===========================================================
#                   Read CSV File 
#===========================================================
genomes_df= pd.read_csv('overview-test.csv')

#===========================================================
#                   Initializing Variables
#===========================================================

#CNk=0                             # kth node of the cluster 
#Memk=0                            # Accessible memory size of
                                   # Switch time between two jobs for CNk
n=len(genomes_df.GenomeName)       # The number of genomes of the reference genome
Gen= genomes_df.GenomeName         # 
Gen= Gen[0:n]                      # The set of genomes of the reference genome
Si=genomes_df.Size                 # 
Si= Si[0:n]                        # Size of gi
J = n                              # The maximum number of Jobs 
poly1=[]
poly2=[]
poly3=[]
poly4=[]


Dictionary={}
FinalOutput={}
#===========================================================
#                   Initializing Genetic Algorithm parameters 
#===========================================================

solution =[]

population_number=20
MutationRate= 0.4
CrossoverRate= 1 - MutationRate
maxIter = 120
MutationNumber = round (population_number* MutationRate);
CrossoverNumber = round (population_number * CrossoverRate);


In [116]:
#================================================================================================
#                  f(X)= aX+b
#================================================================================================
           
def f1 (x, coefficient):
    t=(x*coefficient[0]+coefficient[1])
    return t


#================================================================================================
#                  f(X)= aX^2 + bX + c
#================================================================================================

def f2 (x, coefficient):
    t=(x**2*coefficient[0] +x*coefficient[1]+coefficient[2])
    return t



#================================================================================================
#                  f(X)= aX^3 + bX^2 + cX + d
#================================================================================================
def f3 (x, coefficient):
    t= x**3*coefficient[0] +x**2*coefficient[1] +x*coefficient[2]+coefficient[3]
    return t


#================================================================================================
#                  f(X)= aX^4 + bX^3+ cX^2 + dX + e
#================================================================================================
def f4 (x, coefficient):
    t=(x**4*coefficient[0] +x**3*coefficient[1] +x**2*coefficient[2] +x*coefficient[3]+coefficient[4])
    return t
#================================================================================================
#                  fk(X)
#================================================================================================
def fk (k,x, coefficients):
    if k==1:
        return f1(x, coefficients)
    if k==2:
         return f2(x, coefficients)
    if k==3:
         return f3(x, coefficients)
    if k==4:
         return f4(x, coefficients)


#================================================================================================
#                  Find The Best or Simplest Polynomaial Function Based on Mean Square Error 
#================================================================================================

Estimated_value_by_degree_1 =[]
Estimated_value_by_degree_2 =[]
Estimated_value_by_degree_3 =[]
Estimated_value_by_degree_4 =[]
MSE=[]
BestPloynomialDegree=[[]]
for i in range (V):
  MSE=[]
  #print('i : ', i)
  a= data ==nodename_item[i]
  nodenamep= nodename[a[:,0]]
  xp= x[a[:,0]]
  yp= y[a[:,0]]
  #print('Original X for node name of {} is: {}'.format(int(nodenamep[0]),xp))
  #print('Original Y for node name of {} is: {}'.format(int(nodenamep[0]),yp))
  #print('\n')

  #======================================================
  # Polynomaial Equation Degree of 1  & MSE 
  #======================================================
  coefficient1= np.polyfit(xp,yp,1);
  poly1.append(coefficient1)
  #print('coefficient1= ',coefficient1)
  Estimated_y_by_degree_1 = f1(xp,coefficient1);
  #print('Estimated Y by Degree of 1= ',  Estimated_y_by_degree_1)
  MSE.append(mean_squared_error(yp,Estimated_y_by_degree_1))


  #======================================================
  # Polynomaial Equation Degree of 2 & MSE
  #======================================================
  if (len(xp)>1):
      coefficient2= np.polyfit(xp,yp,2);
      poly2.append(coefficient2)
      #print('coefficient2= ',coefficient2)
      Estimated_y_by_degree_2=f2(xp,coefficient2);
      #print('Estimated Y by Degree of 2= ', Estimated_y_by_degree_2)
      MSE.append( mean_squared_error(yp,Estimated_y_by_degree_2))


  #======================================================
  # Polynomaial Equation Degree of 3 & MSE
  #======================================================
  if (len(xp)>2):
      coefficient3= np.polyfit(xp,yp,3)
      poly3.append(coefficient3)
      #print('coefficient3= ',coefficient3);
      Estimated_y_by_degree_3=f3(xp,coefficient3)
      #print('Estimated Y by Degree of 3= ',Estimated_y_by_degree_3 )
      MSE.append( mean_squared_error(yp,Estimated_y_by_degree_3))


  #======================================================
  # Polynomaial Equation Degree of 4 & MSE
  #======================================================
  if (len(xp)>3):
      coefficient4= np.polyfit(xp,yp,4)
      poly4.append(coefficient4);
      #print('coefficient4= ',coefficient4)
      Estimated_y_by_degree_4= f4(xp,coefficient4)
      #print('Estimated Y by Degree of 4= ',Estimated_y_by_degree_4)
      MSE.append( mean_squared_error(yp,Estimated_y_by_degree_4))


  #======================================================
  # Find Minimum of MSE and its index
  #======================================================
  #print('\n')
  #print ("MSE = ", MSE)
  MinMSE =min(MSE)
  #print("Min(MSE) = ", MinMSE)
  Dictionary={}
  Dictionary["NodeName"] = int( nodenamep[0])
  Dictionary["BestPloynomialDegree"]=MSE.index(MinMSE)+1
  if (MSE.index(MinMSE)==0):
      Dictionary["Coefficients"]=coefficient1
  elif (MSE.index(MinMSE)==1):
      Dictionary["Coefficients"]=coefficient2
  elif (MSE.index(MinMSE)==2):
      Dictionary["Coefficients"]=coefficient3
  elif (MSE.index(MinMSE)==3):
      Dictionary["Coefficients"]=coefficient4
        
  FinalOutput[i]= Dictionary
  BestPloynomialDegree.append([nodenamep[0],MSE.index(MinMSE)+1])

  #print('The Best and simplest Ploynomial Degree for node name {} is: {}'.format(int(nodenamep[0]),MSE.index(MinMSE)+1))
  #print('\n')
  #print ('*****************************************************************************************')
  #print('\n')
#======================================================
# Print result for Best Polynomaial Equation Degree Based MSE
#======================================================
#print('\n')
#print('==========================================================================================')
#print('           Best Ploynomial Degree (Based on MSE): \n', BestPloynomialDegree)
#print('==========================================================================================')
#print('\n')
#print('\n')
#print('\n')

FinalOutput






{0: {'NodeName': 201,
  'BestPloynomialDegree': 4,
  'Coefficients': array([-8.33333333e-04,  2.16666667e-02, -2.29166667e-01,  1.36833333e+00,
         -9.80000000e-01])},
 1: {'NodeName': 202,
  'BestPloynomialDegree': 4,
  'Coefficients': array([-8.33333333e-04,  2.00000000e-02, -1.89166667e-01,  9.90000000e-01,
         -7.40000000e-01])},
 2: {'NodeName': 203,
  'BestPloynomialDegree': 4,
  'Coefficients': array([-4.16666667e-04,  1.08333333e-02, -1.14583333e-01,  6.84166667e-01,
         -4.90000000e-01])}}

In [117]:
def jobs2cluster(S):
    AssignedJobs2Cluster={}
    for i in range (V):
        AssignedJobs2Cluster[i]=[]
    for j in range (n):
        Jobj=int(S[0][j])
        CNk=int(S[1][j])
        AssignedJobs2Cluster[CNk].append(Jobj)
    return (AssignedJobs2Cluster)

In [118]:
def genome2cluster(SS):
    g2c={}
    for i in range (V):
        g2c[i]=[]
    for j in range (n):
        Genomej=j
        CNk=int(SS[1][j])
        g2c[CNk].append(Genomej)
    return(g2c.copy())

In [119]:
"""Feasibility function for the Memk. Returns True if feasible False
    otherwise."""
def feasibleMemk(sol):
 
    
    M=np.zeros((V,J))

        
   
    for i in range (n):
        jobj= int(sol[0][i])
        CNk = int(sol[1][i])
        
        item= Si[i]
        q= len(item)
        if item[q-1]== 'K' or item[q-1]== 'k' :
            DigitSize=item[:-1]
            
            M[CNk][jobj]+= (float(DigitSize)/1000)
        elif  item[q-1]== 'M' or item[q-1]== 'm' :
            DigitSize=item[:-1]
            M[CNk][jobj]+= (float(DigitSize))
        
    for k in range (V):
        for j in range (J):
            if  Memory[k]< 3*M[k][j]:
                return False
    return True
    
  

In [120]:
def ComputeJobSize(g2j):
    sm=[]
    for j in range (J):
            sm.append(0.0)
    for j in range (J):
            GenomesNumber= g2j[j]
            m= len(GenomesNumber)
            
            #extract memory size of each genome belonged to Job j
            MemoryItems = (Si[GenomesNumber])
            
            for i in GenomesNumber:
                item= Si[i]
                q= len(item)
                if item[q-1]== 'K' or item[q-1]== 'k' :
                    DigitSize=item[:-1]
                    sm[j]+= float(DigitSize)/1000
                elif  item[q-1]== 'M' or item[q-1]== 'm' :
                    DigitSize=item[:-1]
                    sm[j]+= float(DigitSize)
    return sm.copy()

In [121]:
def RunningTime (Sl,g2cn):
    tk= np.zeros((V,1))
    J2C=jobs2cluster(Sl)
    G2J=Genomes2Jobs(Sl)
    JS= ComputeJobSize(G2J) # Job Size
    for k in range (V):
        NeedMemory=0
        for j in range (len(np.unique(J2C[k]))):
            #print ('Cluster Number is: {},  Job Number is {}, and Job size is {}. \n '.format (k,J2C[k][j],JS[J2C[k][j]]))
            NeedMemory=JS[J2C[k][j]]
            tk[k]+=fk(FinalOutput[k]['BestPloynomialDegree'],NeedMemory,FinalOutput[k]['Coefficients'])
    for i in range (V):
        GenomesNumber= g2cn[i]
        m= len(np.unique(J2C[i]))
        tk[i]+= ((m-1)*tsk[i])
        
    return max(tk)
            
 
          
        
    
       

In [122]:
def Genomes2Jobs(SL):
    Ge2Jo={}
    for i in range (n):
        Ge2Jo[i]=[]
    for j in range (n):
        Genomej=j
        Jobj=int(SL[0][j])
        Ge2Jo[Jobj].append(Genomej)
    return(Ge2Jo.copy())

In [123]:
def clusters2job(solu):
    C2Job={}
    for i in range (n):
        C2Job[i]=[]
    for j in range (n):
        Jobj=int(solu[0][j])
        CNk=int(solu[1][j])
        C2Job[Jobj].append(CNk)
      
    return(C2Job.copy())


In [124]:
import random
def AddressJobs2ClustersProblem(SOL):
    Temp= clusters2job(SOL.copy())
    t= len(Temp)
    for j in range (len(Temp)):
        if len(Temp[j])>1:
            newCNkIndex= random.randint(0,len(Temp[j])-1)
            newCNk= Temp[j][newCNkIndex]
            for i in range (n):
                if SOL[0][i]== j:
                    SOL[1][i]=newCNk
    return SOL.copy()
                
           
        
    
    
    


In [125]:
def equalSols(S1,S2):
    for j in range(len(S1)):
        if (S1[1][j]!= S2[1][j] or S1[0][j]!= S2[0][j] ):
                return False
    return True

In [126]:
def isRepeated(POp,Sol):
    index= False
    newPop=[]
    for i in range(len(POp)):
        newPop.append((POp[i].copy()))
    newPop=np.array(newPop.copy())
    N= len(POp)
    for i in range(N):
        if (equalSols(Sol.copy(),newPop[i,:,:].copy())):
                index =True
    return index

In [127]:
def arrangeSolutions(POPU):
    newFit = []
    newPop=np.empty((len(POPU), 2,n))
    for i in range(len(POPU)):
         if (feasibleMemk(POPU[i].copy())==True):
           
            newPop[i]=((POPU[i].copy()))
            SolvedSol= AddressJobs2ClustersProblem(POPU[i].copy())   
            G2CNK= genome2cluster(SolvedSol.copy())
            Fit= RunningTime(SolvedSol.copy(),G2CNK)
            newFit.append(Fit)
    
    return newPop.copy(),newFit.copy()
        
    

In [128]:
def AddNewSol(POPulation,Sol,FITS,Fit):
    index= isRepeated(POPulation.copy(),Sol.copy())
    L= len( POPulation.copy())
    if(index == False ):
        L=L+1
        POPulation= np.append(POPulation.copy(),Sol.copy())
        POPulation= POPulation.reshape(L,2,n)
        FITS.append(Fit)
        
    return POPulation.copy(),FITS.copy()

In [129]:
def SortPopulation(poP, FS,Npop):
    
    newFit = []    
    newPop=[]
    
    for i in range(len(poP)):
        newPop.append((poP[i].copy()))
        newFit.append(FS[i][0].copy())
    s = np.array(newFit.copy())
   
    sort_index = np.argsort(newFit.copy())
    for i in range(len(poP)):
        newPop[i]=list( poP[sort_index[i]].copy())
    FS.sort()
    
    return newPop.copy(),FS.copy(),len(newPop.copy())

In [130]:

def CreatePopulation (n):
    #population[i][j][k] ==> i th population 
    #[row0] ==> Job of solution [.......]
    #[row1] ==> CNk of solution [.......]
    Npop=1
    NS= np.zeros((2,n))
    for i in range (n):
            NS[0,i]=int(random.randint(0,J-1))# Random Job  number
            NS[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
    NS= AddressJobs2ClustersProblem(NS.copy())   
    Sol= genome2cluster(NS.copy())
        
    #JobSize = totalMemk(Sol)
    if (feasibleMemk(NS.copy())==True):
            POPULATION= np.empty((1, 2,n))
            for i in range(Npop):
                 POPULATION[0]= NS.copy()
            Fit= RunningTime(NS.copy(),Sol)
    else:
            while(feasibleMemk(NS.copy())==False):
                NS= np.zeros((2,n))
                for i in range (n):
                    NS[0,i]=int(random.randint(0,J-1))# Random Job  number
                    NS[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
                NS= AddressJobs2ClustersProblem(NS.copy())   

                Sol= genome2cluster(NS.copy())

                #JobSize = totalMemk(Sol)
                
                
                if (feasibleMemk(NS.copy())==True):
                    
                    POPULATION= np.empty((1, 2,n))
                    for j in range(n):
                        POPULATION[0]= NS.copy()
                    Fit= RunningTime(NS.copy(),Sol)
            
    return POPULATION.copy(),Fit.copy()
        
    

In [131]:
def Mutation(PP,FITS):   
    P=PP.copy()
    FF=FITS.copy()
    for iMut in range(MutationNumber):
            Npop= len(PP)
            SelectedSolutionIndex= random.randint(0,len(P)-1)
            SelectedSolution= (P[SelectedSolutionIndex]).copy()
            #print("SelectedSolution:\n", SelectedSolution)

            SelectedGenomeIndex1 = random.randint(0,n-1)
            SelectedGenomeIndex2 = random.randint(0,n-1)

            tempJ=SelectedSolution[0][SelectedGenomeIndex1]
            tempCNk=SelectedSolution[1][SelectedGenomeIndex1]
            SelectedSolution[0][SelectedGenomeIndex1]= SelectedSolution[0][SelectedGenomeIndex2].copy()
            SelectedSolution[1][SelectedGenomeIndex1]= SelectedSolution[1][SelectedGenomeIndex2].copy()
            SelectedSolution[0][SelectedGenomeIndex2]= tempJ
            SelectedSolution[1][SelectedGenomeIndex2]= tempCNk
            MutedSolution = AddressJobs2ClustersProblem(SelectedSolution.copy())
            Sol= genome2cluster(MutedSolution.copy())
            
           


            if (feasibleMemk(MutedSolution.copy())==True and  isRepeated(P.copy(),MutedSolution.copy()) ==False):
                #print('Mut len:',len(MutedSolution))
                F=( RunningTime(MutedSolution.copy(),Sol.copy()))
                P, FF= AddNewSol(P.copy(), MutedSolution.copy(),FITS.copy(),F) 
                if len(P)!= len(FF):
                    P,F= arrangeSolutions(P)
                    #print("Pop:\n", PP)
                    #print("!!!Mutelen(P):",len(P))
                    #print ("len Fit",len(F))
    return P, FF



In [132]:
def Crossover (POP,FITN):
    PoP=POP.copy()
    FiT=FITN.copy()
    for iCross in range(CrossoverNumber):
            Npop= len(PoP)
            SelectedSolutionIndex1= random.randint(0,len(PoP)-1)
            SelectedSolutionIndex2= random.randint(0,len(PoP)-1)

            SS1= PoP[SelectedSolutionIndex1].copy()
            SS2= PoP[SelectedSolutionIndex2].copy()

            SelectedGenomeIndex = random.randint(0,n-1)
            temp=  SS2[SelectedGenomeIndex:n-1].copy()
            SS2[SelectedGenomeIndex:n-1]= SS1[SelectedGenomeIndex:n-1].copy()
            SS1[SelectedGenomeIndex:n-1]= temp.copy()


            SS1=AddressJobs2ClustersProblem(SS1.copy())
            Sol1= genome2cluster(SS1.copy())



            #JobSize = totalMemk(Sol1)
            if (feasibleMemk(SS1)==True and isRepeated(PoP.copy(),SS1.copy()) ==False):
                #print('Cross1 len:',len(SS1))
                F=(RunningTime(SS1.copy(),Sol1.copy()))
                PoP,FiT= AddNewSol(PoP.copy(), SS1.copy(),FiT.copy(),F)  
                


            SS2=AddressJobs2ClustersProblem(SS2)
            Sol2= genome2cluster(SS2)
            #JobSize = totalMemk(Sol2)
            if (feasibleMemk(SS2.copy())==True and isRepeated(PoP.copy(),SS2.copy()) ==False):
                #print('Croos2 len:',len(SS2))
                F=(RunningTime(SS2.copy(),Sol2.copy()))
                PoP,FiT= AddNewSol(PoP.copy(), SS2.copy(),FiT.copy(),F) 
    if (len(PoP)!= len(FiT)):
        PoP,FiT= arrangeSolutions(PoP.copy())
        #print("Pop:\n", PP)
        #print("!!!Crosslen(P):",len(PoP))
        #print ("len Fit",len(FiT))
    return PoP.copy(),FiT.copy()

In [133]:
#==================================================================
#                 Genetic Algorithm 
#==================================================================
    
def GA_Scheduling():
    #==================================================================
    #                 Create Initial Population
    #===================================================================
    Population=[]
    MaxPopN= 1000
    Npop=0  # number of added acceptable solutions to population
    fits=[]

    Population,Fit= CreatePopulation (n)
    fits.append(Fit)
    print('initial Population',Population)
    while (Npop <population_number):
        for j in range (population_number):
            NSOL= np.zeros((2,n))
            for i in range (n):
                NSOL[0,i]=int(random.randint(0,J-1))# Random Job  number
                NSOL[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
            NSOL= AddressJobs2ClustersProblem(NSOL)   

            Sol= genome2cluster(NSOL)
            #JobSize = totalMemk(Sol)
            if (feasibleMemk(NSOL)==True and isRepeated(Population.copy(),NSOL.copy()) ==False):
                Fit= (RunningTime(NSOL,Sol))
                Population, fits= AddNewSol(Population.copy(),NSOL.copy(),fits.copy(), Fit)
                Npop+=1
    #print('initpop:',Population)
   
    #==================================================================
    #                 Genetic Algorithm 
    #===================================================================
    for iteration in range(maxIter):
        changed=False

        changed=False
        nMut=0   # number of acceptable Muted solutions 
        nCross=0 # number of acceptable Crossover solutions 
        Npop= len(Population)

        #for i in range (len (Population)):
            #print(Population[i])
        #==================================================================
        #                 Mutation
        #===================================================================
                
        #print ('Mut1:')
        #print(len(Population))
        #print(len(fits))
        Population,fits= Mutation(Population.copy(),fits.copy())
        #print ('Mut2:')
        #print(len(Population))
        #print(len(fits))
        #for i in range (len (Population)):
            #print(Population[i])
        #==================================================================
        #                Crossover
        #===================================================================
        #print ('Cros1:')
        #print(len(Population))
        #print(len(fits))
        Population,fits=Crossover (Population.copy(),fits.copy())
        #print ('Cros2:')
        #print(len(Population))
        #print(len(fits))

        #==================================================================
        #                Add Muted Solutions to Population
        #===================================================================

        ''' print ('Pop1:')
        for i in range (len (Population)):
            print(Population[i])'''
        Population, fits,Npop=SortPopulation(Population.copy(), fits.copy(), Npop-1)    
        Population,fits= arrangeSolutions(Population.copy())
        
        '''print ('Pop2:')
        for i in range (len (Population)):
            print(Population[i])'''
        print("\n***************************************************************************\n")
        print('Best Solution is of iteration of {} is:\n{} '.format(iteration,Population[0].copy() ))
        print("-----------------------------------------------------------------------------\n")
    
    return Population.copy(), fits.copy()

P, F= GA_Scheduling()

initial Population [[[2. 3. 1. 4. 3.]
  [1. 0. 1. 0. 0.]]]

***************************************************************************

Best Solution is of iteration of 0 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 1 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 2 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 3 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


****


***************************************************************************

Best Solution is of iteration of 35 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 36 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 37 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 38 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***********************************************************


***************************************************************************

Best Solution is of iteration of 70 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 71 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 72 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 73 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***********************************************************


***************************************************************************

Best Solution is of iteration of 105 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 106 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 107 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


***************************************************************************

Best Solution is of iteration of 108 is:
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]] 
-----------------------------------------------------------------------------


*******************************************************

In [134]:
BestSolution = P[0]
ClusterName= np.unique(nodename)
print('The best solution:\n')
for i in range(n):
    j= int(BestSolution[0][i])
    k= int(BestSolution[1][i])
    print( "Genome {} is allocated to Job Number {} on Cluster Node Number {}.".format(i,j,ClusterName[k]))

print('\n and its fitness function is:\n',(F[0]))

The best solution:

Genome 0 is allocated to Job Number 3 on Cluster Node Number 202.0.
Genome 1 is allocated to Job Number 0 on Cluster Node Number 202.0.
Genome 2 is allocated to Job Number 0 on Cluster Node Number 202.0.
Genome 3 is allocated to Job Number 0 on Cluster Node Number 202.0.
Genome 4 is allocated to Job Number 1 on Cluster Node Number 203.0.

 and its fitness function is:
 [0.79]


In [135]:
S1=P[0]
print(feasibleMemk(S1))

print(S1)

True
[[3. 0. 0. 0. 1.]
 [1. 1. 1. 1. 2.]]


In [136]:

J2C= jobs2cluster(BestSolution)
EmptyClusters=[]
for k in range(V):
    if (J2C[k]!= []):
        print('Cluster {} includes a subset of Job numbers:\n{}'.format(ClusterName[k],np.unique( J2C[k])))
    else:
        EmptyClusters.append(ClusterName[k])
print("================================================================================")
print ("Empty Clusters are:", (EmptyClusters))

Cluster 202.0 includes a subset of Job numbers:
[0 3]
Cluster 203.0 includes a subset of Job numbers:
[1]
Empty Clusters are: [201.0]


In [137]:

G2J= Genomes2Jobs(BestSolution)
EmptyJobs=[]
for i in range(J):
    if (G2J[i]!= []):
        print('Job {} includes a subset of genomes numbers:{}'.format(i,G2J[i]))
    else:
        EmptyJobs.append(i)
print("================================================================================")
print ("Empty Jobs are:", EmptyJobs)

Job 0 includes a subset of genomes numbers:[1, 2, 3]
Job 1 includes a subset of genomes numbers:[4]
Job 3 includes a subset of genomes numbers:[0]
Empty Jobs are: [2, 4]


In [138]:
job0size=200+50
job0clusternodeIndex=202-201
job1size= 666
job1clusternodeIndex=203-201
job2size=0
job2clusternodeIndex=[]
job3size= 0
job3clusternodeIndex= []
job4size= 400+100
job4clusternodeIndex= 202-201

CN0JobsSize= 0
CN1JobsSize= job0size+ job4size
CN2JobsSize= job1size

FkCN0= fk(FinalOutput[0]['BestPloynomialDegree'],0,FinalOutput[0]['Coefficients'])
FkCN1= fk(FinalOutput[1]['BestPloynomialDegree'],job0size,FinalOutput[1]['Coefficients'])
+fk(FinalOutput[1]['BestPloynomialDegree'],job4size,FinalOutput[2]['Coefficients'])

FkCN2= fk(FinalOutput[2]['BestPloynomialDegree'],job1size,FinalOutput[2]['Coefficients']) 
print('FkCN0:',FkCN0)
print('FkCN1:',FkCN1)
print('FkCN2:',FkCN2)
tk0=FkCN0
tk1=FkCN1
tk2= FkCN2 + tsk[2]
print('tk0:',tk0)
print('tk1:',tk1)
print('tk2:',tk2)
print ('total tk:', max(tk0,tk1,tk2))


FkCN0: -0.9800000000000257
FkCN1: -2954284.4900001213
FkCN2: -78825914.56000137
tk0: -0.9800000000000257
tk1: -2954284.4900001213
tk2: -78825913.56000137
total tk: -0.9800000000000257


In [139]:
print(len(F))
print(len(P))
for i in range(len(P)):
    print('Jobs and for Population {} is:\t{}, {} fits={}'.format(i,P[i][0],P[i][1],F[i]))

190
190
Jobs and for Population 0 is:	[3. 0. 0. 0. 1.], [1. 1. 1. 1. 2.] fits=[0.79]
Jobs and for Population 1 is:	[3. 1. 0. 0. 0.], [1. 2. 1. 1. 1.] fits=[1.]
Jobs and for Population 2 is:	[0. 3. 0. 0. 0.], [0. 2. 0. 0. 0.] fits=[1.]
Jobs and for Population 3 is:	[3. 0. 1. 0. 0.], [0. 1. 2. 1. 1.] fits=[1.29]
Jobs and for Population 4 is:	[0. 0. 3. 0. 0.], [0. 0. 2. 0. 0.] fits=[1.29]
Jobs and for Population 5 is:	[3. 4. 4. 0. 0.], [0. 1. 1. 2. 2.] fits=[1.91]
Jobs and for Population 6 is:	[3. 0. 0. 0. 1.], [2. 0. 0. 0. 2.] fits=[2.29]
Jobs and for Population 7 is:	[0. 1. 0. 3. 0.], [1. 2. 1. 0. 1.] fits=[2.32]
Jobs and for Population 8 is:	[0. 0. 1. 3. 0.], [1. 1. 2. 0. 1.] fits=[2.32]
Jobs and for Population 9 is:	[0. 4. 4. 3. 0.], [2. 1. 1. 0. 2.] fits=[2.32]
Jobs and for Population 10 is:	[3. 4. 2. 2. 0.], [2. 2. 1. 1. 0.] fits=[2.5]
Jobs and for Population 11 is:	[3. 1. 0. 0. 0.], [2. 2. 1. 1. 1.] fits=[2.5]
Jobs and for Population 12 is:	[0. 3. 0. 0. 1.], [1. 1. 1. 1. 2.] fits=[

Jobs and for Population 135 is:	[3. 1. 4. 4. 3.], [2. 0. 0. 0. 2.] fits=[5.98]
Jobs and for Population 136 is:	[0. 4. 2. 2. 3.], [1. 2. 1. 1. 1.] fits=[5.99]
Jobs and for Population 137 is:	[3. 4. 2. 2. 0.], [1. 2. 1. 1. 1.] fits=[5.99]
Jobs and for Population 138 is:	[4. 4. 0. 3. 2.], [0. 0. 1. 1. 1.] fits=[6.09]
Jobs and for Population 139 is:	[2. 3. 1. 4. 3.], [1. 0. 1. 0. 0.] fits=[6.12]
Jobs and for Population 140 is:	[2. 4. 0. 3. 4.], [2. 0. 2. 0. 0.] fits=[6.12]
Jobs and for Population 141 is:	[1. 3. 2. 4. 3.], [1. 0. 1. 0. 0.] fits=[6.12]
Jobs and for Population 142 is:	[1. 2. 0. 4. 2.], [2. 0. 1. 0. 0.] fits=[6.12]
Jobs and for Population 143 is:	[1. 0. 3. 0. 0.], [2. 0. 0. 0. 0.] fits=[6.18]
Jobs and for Population 144 is:	[0. 2. 1. 2. 4.], [1. 1. 1. 1. 0.] fits=[6.22]
Jobs and for Population 145 is:	[4. 0. 1. 1. 4.], [1. 1. 1. 1. 1.] fits=[6.4]
Jobs and for Population 146 is:	[0. 4. 4. 1. 1.], [1. 1. 1. 1. 1.] fits=[6.45]
Jobs and for Population 147 is:	[4. 2. 2. 0. 3.], [1.

In [140]:
print(Si)

0    2M
1    4M
2    6M
3    5M
4    3M
Name: Size, dtype: object


# 